### **This Notebook Detects Representative Spectra within each Surface**

This is done by building a similarity graph between each spectra, based on the Gaussian Kernel, then detecting the most representative spectra within each surface based on the PageRank centrality.

Import relevant libraries and import raw data

In [1]:
import seaborn as sns
import pandas as pd
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
os_name = os.name

if os_name == 'nt':
    password = os.getenv('NEO4J_Password')
    neo4j_directory = os.getenv('NEO4J_Directory')
else:
    password = os.environ['NEO4J_Password']
    neo4j_directory = os.environ['NEO4J_Directory']

In [3]:
df = pd.read_csv("../../data/current_clean_spectrum.csv")

In [4]:
def normalise(absorbances):
    max_value = np.max(absorbances)
    normalized_absorbances = absorbances / max_value
    return normalized_absorbances

df['Absorbance'] = df.groupby('SpecID')['Absorbance'].transform(lambda x: normalise(x))

In [5]:
#sns.lineplot(data=df, x='WaveNumber', y='Absorbance', hue='Status', errorbar=('ci', False))

In [6]:
sample = df[df['SurID'] == '210526-3']

In [7]:
#sns.lineplot(data=sample, x='WaveNumber', y='Absorbance', hue='Status', errorbar=('ci', False))

In [8]:
df

,SpecID,Seq,WaveNumber,SurID,Status,Absorbance
0,201210-1-00,293,400.22778,201210-1,Normal,0.241429
1,201210-1-00,294,400.91116,201210-1,Normal,0.241086
2,201210-1-00,295,401.59454,201210-1,Normal,0.240729
3,201210-1-00,296,402.27789,201210-1,Normal,0.240359
4,201210-1-00,297,402.96127,201210-1,Normal,0.239978
...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,210526-3,Hyperglycemia,0.060284
6239201,210526-3-09,2338,1797.72200,210526-3,Hyperglycemia,0.064627
6239202,210526-3-09,2339,1798.40550,210526-3,Hyperglycemia,0.069153
6239203,210526-3-09,2340,1799.08890,210526-3,Hyperglycemia,0.073864


In [9]:
pivoted_df = df.pivot(index='SpecID', columns='WaveNumber', values='Absorbance').reset_index()
pivoted_df.columns.name = None

statuses = df[['SpecID', 'Status']].drop_duplicates()

pivoted_df = pd.merge(pivoted_df, statuses, on='SpecID')
pivoted_df = pivoted_df.set_index('SpecID')

pivoted_df.head()

,400.22778,400.91116,401.59454,402.27789,402.96127,403.64465,404.32803,405.01138,405.69476,406.37814,...,1794.3053,1794.9886,1795.672,1796.3553,1797.0387,1797.722,1798.4055,1799.0889,1799.7722,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,0.241429,0.241086,0.240729,0.240359,0.239978,0.239588,0.239190,0.238786,0.238378,0.237968,...,0.040039,0.036223,0.032005,0.027369,0.022299,0.016778,0.010790,0.004317,-0.002655,Normal
201210-1-01,0.245468,0.250816,0.255986,0.260980,0.265798,0.270441,0.274912,0.279210,0.283337,0.287294,...,0.033509,0.035876,0.038589,0.041665,0.045119,0.048967,0.053224,0.057908,0.063032,Normal
201210-1-02,0.088417,0.085819,0.083482,0.081397,0.079552,0.077937,0.076541,0.075352,0.074360,0.073553,...,-0.003494,-0.003467,-0.003410,-0.003322,-0.003202,-0.003049,-0.002864,-0.002645,-0.002392,Normal
201210-1-03,0.177716,0.177087,0.176421,0.175722,0.174997,0.174250,0.173488,0.172716,0.171939,0.171163,...,-0.011364,-0.010940,-0.010482,-0.009991,-0.009467,-0.008912,-0.008328,-0.007716,-0.007076,Normal
201210-1-04,0.365204,0.348270,0.332714,0.318491,0.305553,0.293853,0.283344,0.273978,0.265708,0.258488,...,-0.037889,-0.038180,-0.038385,-0.038505,-0.038539,-0.038486,-0.038346,-0.038117,-0.037799,Normal


In [10]:
status_df = df.groupby('SpecID')[['SurID', 'Status']].first()
# surface_and_status.to_csv("../../data/surface_and_status.csv")
status_df

,SurID,Status
SpecID,,
201210-1-00,201210-1,Normal
201210-1-01,201210-1,Normal
201210-1-02,201210-1,Normal
201210-1-03,201210-1,Normal
201210-1-04,201210-1,Normal
...,...,...
210526-3-45,210526-3,Hyperglycemia
210526-3-46,210526-3,Hyperglycemia
210526-3-47,210526-3,Hyperglycemia


>#### **Calculate an adjacency matrix using a Gaussian Kernel**

In [11]:
from scipy.spatial.distance import pdist, squareform

# Drop the 'Status' column as it is not numeric
X = pivoted_df.drop(columns=['Status'])

# Calculate pairwise Euclidean distances
distances = pdist(X.values, metric='euclidean')

# Convert the condensed distances to a square matrix form
distance_matrix = squareform(distances)

In [12]:
def gaussian_kernel(distances, sigma):
    return np.exp(-distances**2 / (2 * sigma**2))

# Sigma is the bandwidth parameter
sigma = 7.5 # Adjust sigma as needed
kernel_matrix = gaussian_kernel(distance_matrix, sigma)

# Convert the kernel matrix to a DataFrame
kernel_df = pd.DataFrame(kernel_matrix, index=pivoted_df.index, columns=pivoted_df.index)

display(kernel_df.head())
kernel_df.shape

SpecID,201210-1-00,201210-1-01,201210-1-02,201210-1-03,201210-1-04,201210-1-05,201210-1-06,201210-1-07,201210-1-09,201210-1-10,...,210526-3-40,210526-3-41,210526-3-42,210526-3-43,210526-3-44,210526-3-45,210526-3-46,210526-3-47,210526-3-48,210526-3-49
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,1.000000,0.764143,0.281686,0.286495,0.606296,0.519218,0.378281,0.435329,0.405590,0.417085,...,0.310547,0.281189,0.409122,0.229566,0.259994,0.230997,0.247458,0.251333,0.247756,0.227466
201210-1-01,0.764143,1.000000,0.175228,0.208799,0.551881,0.451917,0.360502,0.382277,0.392385,0.400121,...,0.298809,0.282623,0.341490,0.235565,0.262716,0.239818,0.259938,0.263003,0.260136,0.227551
201210-1-02,0.281686,0.175228,1.000000,0.459590,0.300859,0.333712,0.207759,0.260044,0.158411,0.116484,...,0.044102,0.037332,0.128337,0.025035,0.027804,0.022815,0.027433,0.031232,0.025547,0.024274
201210-1-03,0.286495,0.208799,0.459590,1.000000,0.231590,0.240418,0.194222,0.249760,0.198125,0.149410,...,0.070242,0.065843,0.143303,0.044841,0.049871,0.042137,0.049151,0.062416,0.051763,0.044914
201210-1-04,0.606296,0.551881,0.300859,0.231590,1.000000,0.896594,0.586029,0.578160,0.468118,0.464506,...,0.270834,0.243554,0.396007,0.196908,0.205074,0.187786,0.216776,0.207194,0.196469,0.192685


(3045, 3045)

**Construct the node and edge import files with the required Admin Import Headers**

In [13]:
relationship_df = kernel_df.reset_index()
relationship_df = relationship_df.melt(id_vars='SpecID', var_name=':END_ID', value_name='Weight:float')
relationship_df = relationship_df.rename(columns={'SpecID': ':START_ID'})
relationship_df[':TYPE'] = 'LINK'

Drop Relationships a node has with itself, along with repeat relationships.

In [14]:
relationship_df = relationship_df[relationship_df[":START_ID"] != relationship_df[":END_ID"]]
relationship_df = relationship_df[relationship_df[":START_ID"] < relationship_df[":END_ID"]]  # Drops redundant relationships
relationship_df

,:START_ID,:END_ID,Weight:float,:TYPE
3045,201210-1-00,201210-1-01,0.764143,LINK
6090,201210-1-00,201210-1-02,0.281686,LINK
6091,201210-1-01,201210-1-02,0.175228,LINK
9135,201210-1-00,201210-1-03,0.286495,LINK
9136,201210-1-01,201210-1-03,0.208799,LINK
...,...,...,...,...
9272019,210526-3-44,210526-3-49,0.876505,LINK
9272020,210526-3-45,210526-3-49,0.843342,LINK
9272021,210526-3-46,210526-3-49,0.880126,LINK
9272022,210526-3-47,210526-3-49,0.828174,LINK


In [15]:
node_df = status_df.reset_index()
node_df.rename(columns={'SpecID':'SpecID:ID', 'Status':'Status:LABEL', 'SurID':'SurID:LABEL'}, inplace=True)
node_df

,SpecID:ID,SurID:LABEL,Status:LABEL
0,201210-1-00,201210-1,Normal
1,201210-1-01,201210-1,Normal
2,201210-1-02,201210-1,Normal
3,201210-1-03,201210-1,Normal
4,201210-1-04,201210-1,Normal
...,...,...,...
3040,210526-3-45,210526-3,Hyperglycemia
3041,210526-3-46,210526-3,Hyperglycemia
3042,210526-3-47,210526-3,Hyperglycemia
3043,210526-3-48,210526-3,Hyperglycemia


In [16]:
node_df.to_csv(f"{neo4j_directory}/import/nodes.csv", index=False)

In [17]:
relationship_df.to_csv(f"{neo4j_directory}/import/relationships.csv", index=False)

In [18]:
from neo4j import GraphDatabase
import os
import pandas as pd
import shutil
import subprocess

---

### **Neo4J Queries**

This moves the nodes and relationships files from data/current_working_graph into the DBMS import folder.

At the moment this uses local paths, but the rest should be automated.

Then run the admin import command.

In [19]:
os_name = os.name

if os_name == 'nt':
    working_dir = f'{neo4j_directory}/bin'
    command = 'neo4j-admin database import full --nodes=import/nodes.csv --relationships=import/relationships.csv --overwrite-destination neo4j'
    result = subprocess.run(command, shell=True, cwd=working_dir, capture_output=True, text=True)
else:
    working_dir = f'{neo4j_directory}'
    command = [
    './bin/neo4j-admin', 'database', 'import', 'full',
    '--nodes=import/nodes.csv',
    '--relationships=import/relationships.csv',
    '--overwrite-destination',  'neo4j'
    ]
    result = subprocess.run(command, cwd=working_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


# Check if the command was successful
if result.returncode == 0:
    print("Import successful")
    print(result.stdout)
else:
    print("Error during import:")
    print(result.stderr)

Import successful
b"Neo4j version: 5.12.0\nImporting the contents of these files into /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/data/databases/neo4j:\nNodes:\n  /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import/nodes.csv\n\nRelationships:\n  /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import/relationships.csv\n\n\nAvailable resources:\n  Total machine memory: 13.49GiB\n  Free machine memory: 3.310GiB\n  Max heap memory : 910.5MiB\n  Max worker threads: 12\n  Configured max memory: 2.219GiB\n  High parallel IO: true\n\nCypher type normalization is enabled (disable with --normalize-types=false):\n  Property type of 'Weight' normalized from 'float' --> 'double' in /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import/relationships.csv\n\nImport 

In [20]:
if os_name != 'nt':
    command = [
        './bin/neo4j', 'restart'
    ]

    result = subprocess.run(command, cwd=neo4j_directory, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Check if the command was successful
    if result.returncode == 0:
        print("Restart successful")
        print(result.stdout)
    else:
        print("Error during restart:")
        print(result.stderr)

Restart successful
b'Stopping Neo4j........ stopped.\nConfiguration file validation successful (with warnings).\nDirectories in use:\nhome:         /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a\nconfig:       /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/conf\nlogs:         /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/logs\nplugins:      /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/plugins\nimport:       /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import\ndata:         /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/data\ncertificates: /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fd

Run and connect to the Neo4j Database

In [21]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"              # Neo4J username
#Password defined at the top of the notebook

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Ensure you close the driver connection when your program ends
def close_driver():
    driver.close()

**We will calculate the centrality rank of each spectra within it's surface**

In [22]:
surfaces = status_df['SurID'].unique()
surfaces

array(['201210-1', '201210-2', '210114-1', '210114-2', '210120-1',
       '210120-2', '210121-1', '210121-2', '210211-1', '210211-2',
       '210217-1', '210217-2', '210218-1', '210224-1', '210225-1',
       '210225-2', '210303-1', '210303-2', '210304-1', '210304-2',
       '210310-1', '210310-2', '210311-1', '210311-2', '210315-1',
       '210315-2', '210318-1', '210318-2', '210324-1', '210324-2',
       '210325-1', '210325-2', '210331-1', '210401-1', '210407-1',
       '210407-2', '210414-1', '210414-2', '210414-3', '210415-1',
       '210415-2', '210419-1', '210419-2', '210421-1', '210428-1',
       '210428-2', '210428-3', '210429-1', '210504-1', '210504-2',
       '210504-3', '210505-1', '210510-1', '210510-2', '210511-1',
       '210518-1', '210519-1', '210519-2', '210519-3', '210524-1',
       '210526-1', '210526-2', '210526-3'], dtype=object)

In [23]:
pagerank_dfs = []

def project_graph(tx, surface):
    query = """
    CALL gds.graph.project(
      'myGraph', 
      $nodeLabel,
      {
        LINK: {
          orientation: 'UNDIRECTED',
          properties: 'Weight'
        }
      }
    )
    """
    tx.run(query, nodeLabel=surface)

def delete_projection(tx):
    query = """
    CALL gds.graph.drop('myGraph')
    """
    tx.run(query)

def run_pagerank_centrality(tx):
    query = """
    CALL gds.pageRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

for surface in surfaces:
    with driver.session() as session:
        session.execute_write(project_graph, surface)
        pagerank_results = session.execute_read(run_pagerank_centrality)
        pagerank_df = pd.DataFrame(pagerank_results, columns=['name', 'PageRank'])
        pagerank_df['SurfacePageRankRank'] = pagerank_df['PageRank'].rank(method='dense', ascending=False)  # Get the rank of centrality of each spectra within the surface
        pagerank_df['ReverseSurfacePageRankRank'] = pagerank_df['PageRank'].rank(method='dense')  # Get the rank of centrality of each spectra within the surface
        pagerank_dfs.append(pagerank_df)
        session.execute_write(delete_projection)


Unable to retrieve routing information
Transaction failed and will be retried in 0.8882588874517778s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 2.229138047361281s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 4.767965705587981s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 6.6188853776531325s (Unable to retrieve routing information)


In [24]:
close_driver()

In [25]:
pagerank_results = pd.concat(pagerank_dfs).rename(columns={'name':'SpecID'})
pagerank_results

,SpecID,PageRank,SurfacePageRankRank,ReverseSurfacePageRankRank
0,201210-1-47,1.115594,1.0,49.0
1,201210-1-39,1.110905,2.0,48.0
2,201210-1-49,1.108847,3.0,47.0
3,201210-1-46,1.107566,4.0,46.0
4,201210-1-37,1.101252,5.0,45.0
...,...,...,...,...
45,210526-3-04,0.751817,46.0,5.0
46,210526-3-06,0.723411,47.0,4.0
47,210526-3-01,0.565104,48.0,3.0
48,210526-3-32,0.554308,49.0,2.0


##### **Test how different Centrality Ranks affect Predictive Performance**

In [26]:
from sklearn.model_selection import cross_validate, GroupKFold
from sklearn.ensemble import ExtraTreesClassifier

In [27]:
df = df.merge(pagerank_results, on='SpecID')
df

,SpecID,Seq,WaveNumber,SurID,Status,Absorbance,PageRank,SurfacePageRankRank,ReverseSurfacePageRankRank
0,201210-1-00,293,400.22778,201210-1,Normal,0.241429,0.943418,26.0,24.0
1,201210-1-00,294,400.91116,201210-1,Normal,0.241086,0.943418,26.0,24.0
2,201210-1-00,295,401.59454,201210-1,Normal,0.240729,0.943418,26.0,24.0
3,201210-1-00,296,402.27789,201210-1,Normal,0.240359,0.943418,26.0,24.0
4,201210-1-00,297,402.96127,201210-1,Normal,0.239978,0.943418,26.0,24.0
...,...,...,...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,210526-3,Hyperglycemia,0.060284,0.999562,25.0,26.0
6239201,210526-3-09,2338,1797.72200,210526-3,Hyperglycemia,0.064627,0.999562,25.0,26.0
6239202,210526-3-09,2339,1798.40550,210526-3,Hyperglycemia,0.069153,0.999562,25.0,26.0
6239203,210526-3-09,2340,1799.08890,210526-3,Hyperglycemia,0.073864,0.999562,25.0,26.0


In [28]:
def prepare_wavelength_df(df, absorbance_col, status_col='Status'):

    # Pivot the DataFrame to get wavelengths as columns and absorbance values
    wavelength_df = df.pivot(index='SpecID', columns='WaveNumber', values=absorbance_col).reset_index()
    wavelength_df.columns.name = None

    # Merge with the statuses based on SpecID
    # Include the SurID to perform GroupKFold CV
    statuses_and_surface = df[['SpecID', 'SurID', status_col]].drop_duplicates()
    wavelength_df = pd.merge(wavelength_df, statuses_and_surface, on='SpecID')

    # Set SpecID as the index
    wavelength_df = wavelength_df.set_index('SpecID')

    return wavelength_df

In [29]:
def evaluate_extra_trees(df):

    # Set the Surfaces as groups
    groups = df['SurID']
    X = df.drop(['Status', 'SurID'], axis=1)
    y = df['Status']

    # Creating the Extra Trees classifier
    et = ExtraTreesClassifier(random_state=1234)
    
    # Using GroupKFold for classification tasks
    cv = GroupKFold(n_splits=10)

    scores = cross_validate(et, X, y, groups=groups, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])

    # Displaying the results
    print(f"{et.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores['test_accuracy']):.4f} +/- {np.std(scores['test_accuracy']):.4f}")
    print(f"{et.__class__.__name__} Cross-Validation Precision: {np.mean(scores['test_precision_macro']):.4f} +/- {np.std(scores['test_precision_macro']):.4f}")
    print(f"{et.__class__.__name__} Cross-Validation Recall: {np.mean(scores['test_recall_macro']):.4f} +/- {np.std(scores['test_recall_macro']):.4f}")
    print(f"{et.__class__.__name__} Cross-Validation F1-Score: {np.mean(scores['test_f1_macro']):.4f} +/- {np.std(scores['test_f1_macro']):.4f}")

In [30]:
wavelength_df = prepare_wavelength_df(df, 'Absorbance')

In [31]:
wavelength_df

,400.22778,400.91116,401.59454,402.27789,402.96127,403.64465,404.32803,405.01138,405.69476,406.37814,...,1794.9886,1795.672,1796.3553,1797.0387,1797.722,1798.4055,1799.0889,1799.7722,SurID,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,0.241429,0.241086,0.240729,0.240359,0.239978,0.239588,0.239190,0.238786,0.238378,0.237968,...,0.036223,0.032005,0.027369,0.022299,0.016778,0.010790,0.004317,-0.002655,201210-1,Normal
201210-1-01,0.245468,0.250816,0.255986,0.260980,0.265798,0.270441,0.274912,0.279210,0.283337,0.287294,...,0.035876,0.038589,0.041665,0.045119,0.048967,0.053224,0.057908,0.063032,201210-1,Normal
201210-1-02,0.088417,0.085819,0.083482,0.081397,0.079552,0.077937,0.076541,0.075352,0.074360,0.073553,...,-0.003467,-0.003410,-0.003322,-0.003202,-0.003049,-0.002864,-0.002645,-0.002392,201210-1,Normal
201210-1-03,0.177716,0.177087,0.176421,0.175722,0.174997,0.174250,0.173488,0.172716,0.171939,0.171163,...,-0.010940,-0.010482,-0.009991,-0.009467,-0.008912,-0.008328,-0.007716,-0.007076,201210-1,Normal
201210-1-04,0.365204,0.348270,0.332714,0.318491,0.305553,0.293853,0.283344,0.273978,0.265708,0.258488,...,-0.038180,-0.038385,-0.038505,-0.038539,-0.038486,-0.038346,-0.038117,-0.037799,201210-1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210526-3-45,0.136945,0.149457,0.160304,0.169571,0.177342,0.183701,0.188732,0.192521,0.195150,0.196705,...,0.083027,0.089574,0.095972,0.102196,0.108222,0.114026,0.119583,0.124871,210526-3,Hyperglycemia
210526-3-46,0.178789,0.170712,0.162990,0.155644,0.148693,0.142156,0.136055,0.130409,0.125238,0.120562,...,0.026669,0.034132,0.042764,0.052618,0.063750,0.076212,0.090060,0.105347,210526-3,Hyperglycemia
210526-3-47,0.230267,0.232734,0.234179,0.234678,0.234306,0.233140,0.231254,0.228724,0.225626,0.222035,...,0.092497,0.103839,0.115810,0.128406,0.141627,0.155471,0.169935,0.185017,210526-3,Hyperglycemia


In [32]:
evaluate_extra_trees(wavelength_df)

/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ExtraTreesClassifier Cross-Validation Accuracy: 0.5617 +/- 0.1082
ExtraTreesClassifier Cross-Validation Precision: 0.5692 +/- 0.1117
ExtraTreesClassifier Cross-Validation Recall: 0.5774 +/- 0.1469
ExtraTreesClassifier Cross-Validation F1-Score: 0.5412 +/- 0.1257


#### **Only keep the top k most central spectra**

In [33]:
representative_df = df[df['SurfacePageRankRank'] <= 40]
representative_df

,SpecID,Seq,WaveNumber,SurID,Status,Absorbance,PageRank,SurfacePageRankRank,ReverseSurfacePageRankRank
0,201210-1-00,293,400.22778,201210-1,Normal,0.241429,0.943418,26.0,24.0
1,201210-1-00,294,400.91116,201210-1,Normal,0.241086,0.943418,26.0,24.0
2,201210-1-00,295,401.59454,201210-1,Normal,0.240729,0.943418,26.0,24.0
3,201210-1-00,296,402.27789,201210-1,Normal,0.240359,0.943418,26.0,24.0
4,201210-1-00,297,402.96127,201210-1,Normal,0.239978,0.943418,26.0,24.0
...,...,...,...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,210526-3,Hyperglycemia,0.060284,0.999562,25.0,26.0
6239201,210526-3-09,2338,1797.72200,210526-3,Hyperglycemia,0.064627,0.999562,25.0,26.0
6239202,210526-3-09,2339,1798.40550,210526-3,Hyperglycemia,0.069153,0.999562,25.0,26.0
6239203,210526-3-09,2340,1799.08890,210526-3,Hyperglycemia,0.073864,0.999562,25.0,26.0


In [34]:
representative_wavelength_df = prepare_wavelength_df(representative_df, 'Absorbance')
evaluate_extra_trees(representative_wavelength_df)

/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ExtraTreesClassifier Cross-Validation Accuracy: 0.5741 +/- 0.0839
ExtraTreesClassifier Cross-Validation Precision: 0.5631 +/- 0.0934
ExtraTreesClassifier Cross-Validation Recall: 0.5735 +/- 0.1210
ExtraTreesClassifier Cross-Validation F1-Score: 0.5364 +/- 0.0996


#### **Alternatively, try dropping the least central spectra from each Surface**

In [35]:
representative_df = df[df['ReverseSurfacePageRankRank'] >= 10]
representative_df

,SpecID,Seq,WaveNumber,SurID,Status,Absorbance,PageRank,SurfacePageRankRank,ReverseSurfacePageRankRank
0,201210-1-00,293,400.22778,201210-1,Normal,0.241429,0.943418,26.0,24.0
1,201210-1-00,294,400.91116,201210-1,Normal,0.241086,0.943418,26.0,24.0
2,201210-1-00,295,401.59454,201210-1,Normal,0.240729,0.943418,26.0,24.0
3,201210-1-00,296,402.27789,201210-1,Normal,0.240359,0.943418,26.0,24.0
4,201210-1-00,297,402.96127,201210-1,Normal,0.239978,0.943418,26.0,24.0
...,...,...,...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,210526-3,Hyperglycemia,0.060284,0.999562,25.0,26.0
6239201,210526-3-09,2338,1797.72200,210526-3,Hyperglycemia,0.064627,0.999562,25.0,26.0
6239202,210526-3-09,2339,1798.40550,210526-3,Hyperglycemia,0.069153,0.999562,25.0,26.0
6239203,210526-3-09,2340,1799.08890,210526-3,Hyperglycemia,0.073864,0.999562,25.0,26.0


In [36]:
representative_wavelength_df = prepare_wavelength_df(representative_df, 'Absorbance')
evaluate_extra_trees(representative_wavelength_df)

/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ExtraTreesClassifier Cross-Validation Accuracy: 0.5846 +/- 0.0987
ExtraTreesClassifier Cross-Validation Precision: 0.5946 +/- 0.1034
ExtraTreesClassifier Cross-Validation Recall: 0.6054 +/- 0.1462
ExtraTreesClassifier Cross-Validation F1-Score: 0.5606 +/- 0.1214


#### **To Counter the Imbalanced Surface Size We Could Also Try A Percentage Based Cutoff**

Drop **n%** least central spectra from each surface.

In [37]:
# Calculate the nth percentile of SurfacePageRankRank for each SurID
median_ranks = df.groupby('SurID')['SurfacePageRankRank'].transform(lambda x: x.quantile(0.85))

# Keep only the rows with SurfacePageRankRank <= to its group's the chosen percentile
filtered_df = df[df['SurfacePageRankRank'] <= median_ranks]

In [38]:
filtered_df

,SpecID,Seq,WaveNumber,SurID,Status,Absorbance,PageRank,SurfacePageRankRank,ReverseSurfacePageRankRank
0,201210-1-00,293,400.22778,201210-1,Normal,0.241429,0.943418,26.0,24.0
1,201210-1-00,294,400.91116,201210-1,Normal,0.241086,0.943418,26.0,24.0
2,201210-1-00,295,401.59454,201210-1,Normal,0.240729,0.943418,26.0,24.0
3,201210-1-00,296,402.27789,201210-1,Normal,0.240359,0.943418,26.0,24.0
4,201210-1-00,297,402.96127,201210-1,Normal,0.239978,0.943418,26.0,24.0
...,...,...,...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,210526-3,Hyperglycemia,0.060284,0.999562,25.0,26.0
6239201,210526-3-09,2338,1797.72200,210526-3,Hyperglycemia,0.064627,0.999562,25.0,26.0
6239202,210526-3-09,2339,1798.40550,210526-3,Hyperglycemia,0.069153,0.999562,25.0,26.0
6239203,210526-3-09,2340,1799.08890,210526-3,Hyperglycemia,0.073864,0.999562,25.0,26.0


In [39]:
representative_wavelength_df = prepare_wavelength_df(filtered_df, 'Absorbance')
representative_wavelength_df

,400.22778,400.91116,401.59454,402.27789,402.96127,403.64465,404.32803,405.01138,405.69476,406.37814,...,1794.9886,1795.672,1796.3553,1797.0387,1797.722,1798.4055,1799.0889,1799.7722,SurID,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,0.241429,0.241086,0.240729,0.240359,0.239978,0.239588,0.239190,0.238786,0.238378,0.237968,...,0.036223,0.032005,0.027369,0.022299,0.016778,0.010790,0.004317,-0.002655,201210-1,Normal
201210-1-01,0.245468,0.250816,0.255986,0.260980,0.265798,0.270441,0.274912,0.279210,0.283337,0.287294,...,0.035876,0.038589,0.041665,0.045119,0.048967,0.053224,0.057908,0.063032,201210-1,Normal
201210-1-04,0.365204,0.348270,0.332714,0.318491,0.305553,0.293853,0.283344,0.273978,0.265708,0.258488,...,-0.038180,-0.038385,-0.038505,-0.038539,-0.038486,-0.038346,-0.038117,-0.037799,201210-1,Normal
201210-1-05,0.382878,0.373423,0.364905,0.357296,0.350565,0.344685,0.339627,0.335362,0.331861,0.329096,...,-0.039434,-0.039675,-0.039926,-0.040197,-0.040498,-0.040838,-0.041226,-0.041672,201210-1,Normal
201210-1-06,0.315916,0.334725,0.352717,0.369919,0.386358,0.402060,0.417053,0.431363,0.445017,0.458042,...,-0.023537,-0.025721,-0.028208,-0.031020,-0.034177,-0.037699,-0.041608,-0.045924,201210-1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210526-3-45,0.136945,0.149457,0.160304,0.169571,0.177342,0.183701,0.188732,0.192521,0.195150,0.196705,...,0.083027,0.089574,0.095972,0.102196,0.108222,0.114026,0.119583,0.124871,210526-3,Hyperglycemia
210526-3-46,0.178789,0.170712,0.162990,0.155644,0.148693,0.142156,0.136055,0.130409,0.125238,0.120562,...,0.026669,0.034132,0.042764,0.052618,0.063750,0.076212,0.090060,0.105347,210526-3,Hyperglycemia
210526-3-47,0.230267,0.232734,0.234179,0.234678,0.234306,0.233140,0.231254,0.228724,0.225626,0.222035,...,0.092497,0.103839,0.115810,0.128406,0.141627,0.155471,0.169935,0.185017,210526-3,Hyperglycemia


In [40]:
evaluate_extra_trees(representative_wavelength_df)

/home/eddie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ExtraTreesClassifier Cross-Validation Accuracy: 0.5736 +/- 0.1016
ExtraTreesClassifier Cross-Validation Precision: 0.5805 +/- 0.1024
ExtraTreesClassifier Cross-Validation Recall: 0.5886 +/- 0.1433
ExtraTreesClassifier Cross-Validation F1-Score: 0.5514 +/- 0.1198
